In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/home-data-for-ml-course/sample_submission.csv
/kaggle/input/home-data-for-ml-course/sample_submission.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv.gz
/kaggle/input/home-data-for-ml-course/data_description.txt
/kaggle/input/home-data-for-ml-course/test.csv.gz
/kaggle/input/home-data-for-ml-course/train.csv
/kaggle/input/home-data-for-ml-course/test.csv


In [2]:
df=pd.read_csv('/kaggle/input/home-data-for-ml-course/train.csv')

In [3]:
df.columns

Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [4]:
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
df=df.select_dtypes(include=['number'])

In [6]:
corr_matrix = df.corr()
selected_features = corr_matrix["SalePrice"].abs().sort_values(ascending=False)
selected_features = selected_features[selected_features > 0.5].index.tolist()
selected_features.remove("SalePrice")

print(selected_features)

['OverallQual', 'GrLivArea', 'GarageCars', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 'FullBath', 'TotRmsAbvGrd', 'YearBuilt', 'YearRemodAdd']


In [7]:
X=df[selected_features]
y=df.SalePrice

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
rfg=RandomForestRegressor(max_leaf_nodes=100)

In [12]:
rfg=rfg.fit(X_train,y_train)

In [13]:
preds=rfg.predict(X_test)
print(preds[:10])

[226313.41365766 131835.50587873 119061.76724259 236562.06281739
  96770.82921582 119681.44247685 247931.57234554 119100.02474684
 601808.62557143 155183.13383586]


In [14]:
y_test[:10]

529     200624
491     133000
459     110000
279     192000
655      88000
1013     85000
1403    282922
601     141000
1182    745000
687     148800
Name: SalePrice, dtype: int64

In [15]:
from sklearn.metrics import mean_absolute_error

In [16]:
mae=mean_absolute_error(preds,y_test)
mae

18841.294223393157

In [17]:
from xgboost import XGBRegressor
rfg = XGBRegressor(n_estimators=500, learning_rate=0.05)

In [18]:
rfg.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.05, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=500, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [25]:
test_data=pd.read_csv('/kaggle/input/home-data-for-ml-course/test.csv')

In [30]:
X_test=test_data[selected_features]

In [31]:
preds=rfg.predict(X_test)
print(preds[:10])

[124699.78  150303.19  167595.16  180906.81  205232.81  183518.36
 167264.75  171448.47  185288.45  121349.625]


In [21]:
rfg = XGBRegressor(
    n_estimators=1000,  # Increase trees
    learning_rate=0.01,  # Lower learning rate
    max_depth=5,  # Control tree depth
    subsample=0.8,  # Randomly sample data
    colsample_bytree=0.8,  # Randomly sample features
    random_state=42
)
rfg.fit(X_train,y_train)
preds=rfg.predict(X_test)
print(preds[:10])
mae=mean_absolute_error(y_test,preds)
print(mae)


[239635.95 136490.12 117801.73 224349.97  95030.25 113329.96 248892.72
 127991.84 668748.6  147685.84]
18546.04792380137


In [33]:
submission_df = pd.DataFrame({
    "Id": test_data['Id'],   # Use the same Ids as the test dataset
    "SalePrice": preds    # Predicted house prices
})
submission_df.to_csv("submission.csv", index=False)